In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
import src.wdata as wdata
from netCDF4 import Dataset
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num
from sklearn import datasets, svm, metrics
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
def getandclose(ds, var='data'):
    dat = ds['/%s'%var][0] 
    ds.close()
    return dat

ncs = np.array([getandclose(ds) for band in [2,3,4,6] 
                for ds in wdata.Sat.getMonth(2017, 7, band)])
print ncs.shape
bands = np.array([getandclose(ds,'bands') for band in [2,3,4,6] 
                for ds in wdata.Sat.getMonth(2017, 7, band)])
print bands.shape
ncs, bands = shuffle(ncs, bands)

In [ ]:
def runClf(classifier, data, bands, pr=True):
    n_samples = len(bands)
#     scaler = StandardScaler()
#     data = scaler.fit_transform(data)
    classifier.fit(data[:n_samples // 2], bands[:n_samples // 2])
    expected = bands[n_samples // 2:]
#     predicted = classifier.predict(scaler.transform(data[n_samples // 2:]))
    predicted = classifier.predict(data[n_samples // 2:])
    report = metrics.classification_report(expected, predicted)
    if pr: print("Classification report for classifier %s:\n%s\n"
          % (classifier, report))
#     print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
    return float(report.strip().split()[-4]) # precision

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(bands)
data = ncs.reshape((n_samples, -1))

In [ ]:
vm = svm.SVC(degree=6,gamma=0.001)
runClf(vm, data, bands)

In [ ]:
nn = MLPClassifier(solver='adam', alpha=1e-5,
         hidden_layer_sizes=(50,80), random_state=1)
runClf(nn, data, bands)

In [ ]:
from itertools import product
precs = []
for a in [1e-5, 1e-3]:
    for arch in product([10,20,30,40,50,60,70,80,90],[10,20,30,40,50,60,70,80,90]):
        print '-'*20
        print arch, a,':'
        nn = MLPClassifier(solver='adam', alpha=a,
                 hidden_layer_sizes=(arch[0]), random_state=1)
        prec = runClf(nn, data, bands, pr=False)
        precs.append(prec)
        print prec
print sorted(precs)[-1]